#	What is Query Expansion/Transformation RAG?

Query Expansion is a technique used to improve the effectiveness of information retrieval systems by reformulating or augmenting the original query. The goal is to improve the recall of relevant documents by including related terms or concepts that might not have been explicitly mentioned in the original query.

# Query Transformation RAG Implementation:

1. **Query Transformation:** We use the LLM to generate 3 alternative versions of the original query.
2. **Multi-Query Retrieval:** We retrieve documents using both the original query and the transformed queries.
3. **Deduplication:** We remove duplicate documents from the retrieved set.
4. **Response Generation:** Using the combined context from all retrieved documents, we generate a final response to the original query.

# Setup

1. **LLM:** Google's free gemini-pro api endpoint (Google's API Key)
2. **Vector Store:** ChromaDB
3. **Embedding Model:** nomic-embed-text-v1.5
4. **LLM Framework:** LangChain
5. **Huggingface API Key**


# Install required libraries

In [1]:
!pip install -q -U \
     Sentence-transformers==3.0.1 \
     langchain==0.2.11 \
     langchain-google-genai==1.0.7 \
     langchain-chroma==0.1.2 \
     langchain-community==0.2.10 \
     langchain-huggingface==0.0.3 \
     einops==0.8.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 12.1 MB/s et

# Import related libraries related to Langchain, HuggingfaceEmbedding

In [2]:
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import Document
from langchain.document_loaders import WebBaseLoader

In [3]:
import os
import getpass

# Provide Google API Key. You can create Google API key at following link

[Google Gemini-Pro API Creation Link](https://console.cloud.google.com/apis/credentials)

[YouTube Video](https://www.youtube.com/watch?v=ZHX7zxvDfoc)



In [4]:
os.environ["GOOGLE_API_KEY"] = getpass.getpass()

··········


# Provide Huggingface API Key. You can create Huggingface API key at following link

[Higgingface API Creation Link](https://huggingface.co/settings/tokens)




In [5]:
os.environ["HF_TOKEN"] = getpass.getpass()

··········


# Step 1: Load and preprocess data code

In [6]:
def load_and_process_data(url):
    # Load data from web
    loader = WebBaseLoader(url)
    data = loader.load()

    # Split text into chunks (Experiment with Chunk Size and Chunk Overlap to get optimal chunking)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = text_splitter.split_documents(data)

    return chunks

# Step 2: Create multiple vector stores code

In [7]:
def create_vector_store(chunks):
  embeddings = HuggingFaceEmbeddings(model_name="nomic-ai/nomic-embed-text-v1.5", model_kwargs = {'trust_remote_code': True})
  vectorstore = Chroma.from_documents(chunks, embeddings)
  return vectorstore

# Step 3: Query Transformation-RAG related code

1. **Query Transformation:** We use the LLM to generate 3 alternative versions of the original query.
2. **Multi-Query Retrieval:** We retrieve documents using both the original query and the transformed queries.
3. **Deduplication:** We remove duplicate documents from the retrieved set.
4. **Response Generation:** Using the combined context from all retrieved documents, we generate a final response to the original query.

In [8]:
def query_transformation_rag(original_query, vectorstore, llm):
    # Query Transformation
    transform_prompt = ChatPromptTemplate.from_template(
        "Given the original query, generate 3 alternative versions that might improve "
        "retrieval effectiveness. Each version should capture a different aspect or "
        "use different terminology related to the original query.\n"
        "Original query: {query}\n"
        "Transformed queries (provide 3):"
    )
    transform_chain = transform_prompt | llm
    try:
        transformed_queries_response = transform_chain.invoke({"query": original_query})
        transformed_queries = transformed_queries_response.content.split('\n')
        transformed_queries = [q.split(': ')[-1].strip() for q in transformed_queries if q.strip()]
        transformed_queries = transformed_queries[:3]  # Ensure we have at most 3 queries
    except Exception as e:
        print(f"Error transforming query: {e}")
        transformed_queries = [original_query]  # Fallback to original query

    # Retrieve documents for each transformed query
    all_docs = []
    for query in [original_query] + transformed_queries:
        docs = vectorstore.similarity_search(query, k=2)
        all_docs.extend(docs)

    # Remove duplicates and combine retrieved documents
    unique_docs = list({doc.page_content: doc for doc in all_docs}.values())
    context = "\n\n".join([doc.page_content for doc in unique_docs])

    # Generate response using combined context
    response_prompt = ChatPromptTemplate.from_template(
        "You are an AI assistant tasked with answering questions based on the provided context. "
        "The context contains information retrieved using the original query and its transformed versions. "
        "Please analyze the context carefully and provide a comprehensive answer to the original question. "
        "If the context doesn't contain enough information, use your general knowledge to supplement the answer, "
        "but prioritize information from the context when available.\n\n"
        "Original Question: {original_query}\n"
        "Context:\n{context}\n\n"
        "Answer:"
    )
    response_chain = response_prompt | llm
    try:
        response = response_chain.invoke({"context": context, "original_query": original_query})
        final_answer = response.content
    except Exception as e:
        print(f"Error generating response: {e}")
        final_answer = "I apologize, but I encountered an error while generating the response."

    return {
        "original_query": original_query,
        "transformed_queries": transformed_queries,
        "final_answer": final_answer,
        "retrieved_context": context
    }

# Step 4: Create chunk of web data to Chroma Vector Store

In [9]:
# Initialize the gemini-pro language model with specified settings (Change temeprature  and other parameters as per your requirement)
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.3, safety_settings={
          HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        },)

# Load and process data
url = "https://en.wikipedia.org/wiki/Artificial_intelligence"
chunks = load_and_process_data(url)

# Create vector store
vectorstore = create_vector_store(chunks)

modules.json:   0%|          | 0.00/255 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/140 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/71.2k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

# Step 5: Run Query Transformation RAG

This implementation shows the key parts of Query Transformation RAG:

1. Generation of alternative query formulations to capture different aspects or terminology
2. Retrieval using both original and transformed queries to improve coverage
3. Deduplication of retrieved documents to avoid redundancy
Generation of a comprehensive response using the expanded context

In [10]:
# Example query
original_query = "What are the ethical considerations in AI development?"

# Run Query Transformation RAG
result = query_transformation_rag(original_query, vectorstore, llm)

print("Original Query:", result["original_query"])
print("\nTransformed Queries:")
for i, query in enumerate(result["transformed_queries"], 1):
    print(f"{i}. {query}")
print("\nFinal Answer:")
print(result["final_answer"])
print("\nRetrieved Context (first 500 characters):")
print(result["retrieved_context"][:500] + "...")

Original Query: What are the ethical considerations in AI development?

Transformed Queries:
1. 1. **Ethical implications of artificial intelligence advancement**
2. 2. **Moral dilemmas in the creation and deployment of AI systems**
3. Balancing innovation with ethical values**

Final Answer:
**Ethical Considerations in AI Development**

The ethical implications of AI development arise at various stages, including design, development, and implementation. Ethical frameworks like the Care and Act Framework guide AI projects by assessing them in four key areas:

* **Wellbeing:** Ensuring the well-being of individuals and communities affected by AI technologies.
* **Social and Ethical Implications:** Considering the societal and ethical impacts of AI systems throughout their lifecycle.
* **Collaboration:** Fostering collaboration among professionals involved in AI development, such as data scientists, product managers, and domain experts.
* **Long-Term Effects:** Evaluating the potential l